In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

# GroupBy

#### By “group by” we are referring to a process involving one or more of the following steps:
1. Splitting the data into groups based on some criteria.
2. Applying a function to each group independently.
3. Combining the results into a data structure.

#### A list or array of values that is the same length as the axis being grouped
#### A value indicating a column name in a DataFrame
#### A dict or Series giving a correspondence bet the values on the axis being grouped and the group names
#### A function to be invoked on the axis index or the individual labels in the index

In [2]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a','b'], 'key2' : ['one', 'two', 'one', 'two', 'one','one'],
                   'data1' : np.random.randn(6),'data2' : np.random.randn(6)})

In [3]:
df

,key1,key2,data1,data2
0,a,one,-0.876501,0.580068
1,a,two,1.669297,0.688734
2,b,one,-0.152680,-0.116673
3,b,two,-2.101728,-0.080179
4,a,one,0.667030,0.485431
5,b,one,1.232165,-0.476657


### to compute the mean of the data1 column using the labels from key1.

In [4]:
grouped = df['data1'].groupby(df['key1'])

In [5]:
grouped

In [6]:
grouped.mean()

key1
a    0.486608
b   -0.340748
Name: data1, dtype: float64

data (a Series) has been aggregated according to the group key, producinga new Series that is now indexed by the unique values in the key1 column. The result index has the name'key1' because the DataFrame column ```df['key1']```

In [7]:
df.groupby('key1')['data1']

In [8]:
df.groupby('key1')[['data2']]

In [9]:
df['data1'].groupby(df['key1'])

In [10]:
df[['data2']].groupby(df['key1'])

###  column names as the group keys

In [11]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,0.486608,0.584744
b,-0.340748,-0.224503


###  with multiple keys

In [12]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()

In [13]:
means

key1  key2
a     one    -0.104736
      two     1.669297
b     one     0.539742
      two    -2.101728
Name: data1, dtype: float64

In [14]:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one   0.532749
     two   0.688734
b    one  -0.296665
     two  -0.080179

###  Here we grouped the data using two keys, and the resulting Series now has a hierarchical index consisting of the unique pairs of keys

###  multiple column names as the group keys

In [15]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one  -0.104736  0.532749
     two   1.669297  0.688734
b    one   0.539742 -0.296665
     two  -2.101728 -0.080179

###  groupby with series as array with same length

In [16]:
city = np.array(['Hyd', 'Kol', 'Hyd', 'Pune', 'Pune','Kol'])

In [17]:
years = np.array([2016, 2016, 2017, 2016, 2017,2017])

In [18]:
df['data1'].groupby([city, years]).mean()

Hyd   2016   -0.876501
      2017   -0.152680
Kol   2016    1.669297
      2017    1.232165
Pune  2016   -2.101728
      2017    0.667030
Name: data1, dtype: float64

###  Size
### useful GroupBy method , which returns a Series containing group sizes
### any missing values in a group key will be excluded from the result

In [19]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     2
      two     1
dtype: int64

# Iterating Over Groups

### The GroupBy object supports iteration, generating a sequence of 2-tuples
### containing the group name along with the chunk of data

In [20]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one -0.876501  0.580068
1    a  two  1.669297  0.688734
4    a  one  0.667030  0.485431
b
  key1 key2     data1     data2
2    b  one -0.152680 -0.116673
3    b  two -2.101728 -0.080179
5    b  one  1.232165 -0.476657


### with multiple keys

 ### In the case of multiple keys, the first element in the tuple will
### be a tuple of key values

In [21]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2)) 
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one -0.876501  0.580068
4    a  one  0.667030  0.485431
('a', 'two')
  key1 key2     data1     data2
1    a  two  1.669297  0.688734
('b', 'one')
  key1 key2     data1     data2
2    b  one -0.152680 -0.116673
5    b  one  1.232165 -0.476657
('b', 'two')
  key1 key2     data1     data2
3    b  two -2.101728 -0.080179


### groupby computing on dict of the data pieces as a one-liner

In [22]:
pieces = dict(list(df.groupby('key1')))

In [23]:
pieces['b']

,key1,key2,data1,data2
2,b,one,-0.152680,-0.116673
3,b,two,-2.101728,-0.080179
5,b,one,1.232165,-0.476657


### groupby on axis1

In [24]:
grouped = df.groupby(df.dtypes, axis=1)

In [25]:
for dtype, group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0 -0.876501  0.580068
1  1.669297  0.688734
2 -0.152680 -0.116673
3 -2.101728 -0.080179
4  0.667030  0.485431
5  1.232165 -0.476657
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one
5    b  one


# Grouping with Dicts and Series

In [26]:
students = pd.DataFrame(np.random.randn(5, 5),columns=['a', 'b', 'c', 'd', 'e'],
          index=['Ram', 'Shyam', 'Mohan', 'Rohan', 'Sohan'])

In [27]:
students.iloc[2:3, [1, 2]] = np.nan

In [28]:
students

,a,b,c,d,e
Ram,0.441341,-0.491311,0.910335,0.301652,-0.242718
Shyam,-1.495241,-1.301358,-0.751829,0.793290,0.170534
Mohan,0.728771,NaN,NaN,0.040052,2.288785
Rohan,0.959121,-0.994217,-0.208440,1.713167,0.146090
Sohan,0.855071,0.513778,0.155925,0.826492,-1.615115


In [29]:
diction = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f' : 'orange'}

In [30]:
df_dict_grp_by = students.groupby(diction, axis=1)

In [31]:
df_dict_grp_by.sum()

,blue,red
Ram,1.211986,-0.292687
Shyam,0.041461,-2.626066
Mohan,0.040052,3.017556
Rohan,1.504728,0.110993
Sohan,0.982416,-0.246266


### with series

In [32]:
w_series = pd.Series(diction)

In [33]:
w_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [34]:
students.groupby(w_series, axis=1).count()

,blue,red
Ram,2,3
Shyam,2,3
Mohan,1,2
Rohan,2,3
Sohan,2,3


# Grouping with Functions

### Any function passed as a group key
### will be called once per index value, with the return values being
### used as the group names

### Suppose you wanted to group by the length of the names; while you could compute an array of string lengths, it’s simpler to just pass the ```len``` function

In [35]:
students.groupby(len).sum()

,a,b,c,d,e
3,0.441341,-0.491311,0.910335,0.301652,-0.242718
5,1.047722,-1.781798,-0.804345,3.373001,0.990294


### Mixing functions with arrays, dicts, or Series is not a problem as everything gets converted to arrays internally

In [36]:
_list = ['one', 'one', 'one', 'two', 'two']

In [37]:
students.groupby([len, _list]).min()

a         b         c         d         e
3 one  0.441341 -0.491311  0.910335  0.301652 -0.242718
5 one -1.495241 -1.301358 -0.751829  0.040052  0.170534
  two  0.855071 -0.994217 -0.208440  0.826492 -1.615115

# Grouping by Index Levels

In [38]:
col = pd.MultiIndex.from_arrays([['UP', 'UP', 'UP', 'HP', 'HP'],[1, 3, 5, 1, 3]],
                                names=['city', 'centre'])

In [39]:
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=col)

In [40]:
hier_df

city          UP                            HP          
centre         1         3         5         1         3
0      -1.043538  1.007456  0.417077  0.707281  0.657808
1      -1.525570 -1.500516  0.284091 -0.218844 -0.104302
2      -0.506390  0.762381  0.361720  0.146136  1.050306
3       1.642674 -0.522072 -0.407664 -0.833518  0.688045

### To group by level, pass the level number or name using the level keyword

In [41]:
hier_df.groupby(level='city', axis=1).count()

city,HP,UP
0,2,3
1,2,3
2,2,3
3,2,3


# Data Aggregation

 ### Aggregations refer to any data transformation that produces scalar values from arrays

Aggregation: compute a summary statistic (or statistics) for each group. Some examples:
Compute group sums or means.
Compute group sizes / counts.

In [42]:
df

,key1,key2,data1,data2
0,a,one,-0.876501,0.580068
1,a,two,1.669297,0.688734
2,b,one,-0.152680,-0.116673
3,b,two,-2.101728,-0.080179
4,a,one,0.667030,0.485431
5,b,one,1.232165,-0.476657


In [43]:
grouped = df.groupby('key1')

In [44]:
grouped['data1'].quantile(0.9)

key1
a    1.468843
b    0.955196
Name: data1, dtype: float64

### User Defined function

In [45]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

In [46]:
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,2.545798,0.203303
b,3.333893,0.396478


### methods like describe also work, even though they are not aggregations

In [47]:
grouped.describe()

data1                                                             \
     count      mean       std       min       25%      50%       75%   
key1                                                                    
a      3.0  0.486608  1.282453 -0.876501 -0.104736  0.66703  1.168163   
b      3.0 -0.340748  1.674884 -2.101728 -1.127204 -0.15268  0.539742   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     1.669297   3.0  0.584744  0.101732  0.485431  0.532749  0.580068   
b     1.232165   3.0 -0.224503  0.219133 -0.476657 -0.296665 -0.116673   

                          
           75%       max  
key1                      
a     0.634401  0.688734  
b    -0.098426 -0.080179

Inside GroupBy, when you invoke a method like describe, it is actually just a
shortcut of below function

In [48]:
f = lambda x: x.describe()
grouped.apply(f)

data1     data2
key1                          
a    count  3.000000  3.000000
     mean   0.486608  0.584744
     std    1.282453  0.101732
     min   -0.876501  0.485431
     25%   -0.104736  0.532749
     50%    0.667030  0.580068
     75%    1.168163  0.634401
     max    1.669297  0.688734
b    count  3.000000  3.000000
     mean  -0.340748 -0.224503
     std    1.674884  0.219133
     min   -2.101728 -0.476657
     25%   -1.127204 -0.296665
     50%   -0.152680 -0.116673
     75%    0.539742 -0.098426
     max    1.232165 -0.080179

# Column-Wise and Multiple Function Application

In [49]:
tips = sns.load_dataset('tips')

In [50]:
tips.head(2)

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3


### Add tip percentage of total bill

In [51]:
tips['tip_pct'] = tips['tip'] / tips['total_bill']

In [52]:
tips[:5]

,total_bill,tip,sex,smoker,day,time,size,tip_pct
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808


In [53]:
grouped = tips.groupby(['day', 'smoker'])

In [54]:
grouped_pct = grouped['tip_pct']

In [55]:
grouped_pct.agg('mean')

day   smoker
Thur  Yes       0.163863
      No        0.160298
Fri   Yes       0.174783
      No        0.151650
Sat   Yes       0.147906
      No        0.158048
Sun   Yes       0.187250
      No        0.160113
Name: tip_pct, dtype: float64

If you pass a list of functions or function names instead, 
you get back a DataFrame with column names taken from the functions

In [56]:
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Thur Yes     0.163863  0.039389      0.151240
     No      0.160298  0.038774      0.193350
Fri  Yes     0.174783  0.051293      0.159925
     No      0.151650  0.028123      0.067349
Sat  Yes     0.147906  0.061375      0.290095
     No      0.158048  0.039767      0.235193
Sun  Yes     0.187250  0.154134      0.644685
     No      0.160113  0.042347      0.193226

 if you pass a list of (name,function) tuples, 
 the first element of each tuple will be used as the DataFrame column names

In [57]:
grouped_pct.agg([('ele1', 'mean'), ('ele2', np.std)])

ele1      ele2
day  smoker                    
Thur Yes     0.163863  0.039389
     No      0.160298  0.038774
Fri  Yes     0.174783  0.051293
     No      0.151650  0.028123
Sat  Yes     0.147906  0.061375
     No      0.158048  0.039767
Sun  Yes     0.187250  0.154134
     No      0.160113  0.042347

With a DataFrame you can specify a list of functions to apply to all of the columns 
or different functions per column

In [58]:
func = ['count', 'mean', 'max']

In [59]:
result = grouped['tip_pct', 'total_bill'].agg(func)

C:\Users\VRaj\miniconda3\envs\myenv\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


In [60]:
result                    # hierarchical index in o/p

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Thur Yes         17  0.163863  0.241255         17  19.190588  43.11
     No          45  0.160298  0.266312         45  17.113111  41.19
Fri  Yes         15  0.174783  0.263480         15  16.813333  40.17
     No           4  0.151650  0.187735          4  18.420000  22.75
Sat  Yes         42  0.147906  0.325733         42  21.276667  50.81
     No          45  0.158048  0.291990         45  19.661778  48.33
Sun  Yes         19  0.187250  0.710345         19  24.120000  45.35
     No          57  0.160113  0.252672         57  20.506667  48.17

In [61]:
result['tip_pct']

count      mean       max
day  smoker                           
Thur Yes        17  0.163863  0.241255
     No         45  0.160298  0.266312
Fri  Yes        15  0.174783  0.263480
     No          4  0.151650  0.187735
Sat  Yes        42  0.147906  0.325733
     No         45  0.158048  0.291990
Sun  Yes        19  0.187250  0.710345
     No         57  0.160113  0.252672

### list of tuples with custom names

In [62]:
fun_tup = [('Doremon', 'mean'), ('Picachoo', np.var)]

In [63]:
grouped['tip_pct', 'total_bill'].agg(fun_tup)

C:\Users\VRaj\miniconda3\envs\myenv\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


tip_pct           total_bill            
              Doremon  Picachoo    Doremon    Picachoo
day  smoker                                           
Thur Yes     0.163863  0.001551  19.190588   69.808518
     No      0.160298  0.001503  17.113111   59.625081
Fri  Yes     0.174783  0.002631  16.813333   82.562438
     No      0.151650  0.000791  18.420000   25.596333
Sat  Yes     0.147906  0.003767  21.276667  101.387535
     No      0.158048  0.001581  19.661778   79.908965
Sun  Yes     0.187250  0.023757  24.120000  109.046044
     No      0.160113  0.001793  20.506667   66.099980

### Apply potentially different functions to one or more of the columns

we need to pass a dict to agg that contains a mapping of column names to
any of the function specified

In [64]:
grouped.agg({'tip' : np.max, 'size' : 'sum'})

tip  size
day  smoker             
Thur Yes      5.00    40
     No       6.70   112
Fri  Yes      4.73    31
     No       3.50     9
Sat  Yes     10.00   104
     No       9.00   115
Sun  Yes      6.50    49
     No       6.00   167

A DataFrame will have hierarchical columns only if multiple
functions are applied to at least one column

In [65]:
grouped.agg({'tip_pct' : ['min', 'max', 'mean', 'std'],
             'size' : 'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Thur Yes     0.090014  0.241255  0.163863  0.039389   40
     No      0.072961  0.266312  0.160298  0.038774  112
Fri  Yes     0.103555  0.263480  0.174783  0.051293   31
     No      0.120385  0.187735  0.151650  0.028123    9
Sat  Yes     0.035638  0.325733  0.147906  0.061375  104
     No      0.056797  0.291990  0.158048  0.039767  115
Sun  Yes     0.065660  0.710345  0.187250  0.154134   49
     No      0.059447  0.252672  0.160113  0.042347  167

# Returning Aggregated Data Without Row Indexes

the aggregated data comes back with
an index, potentially hierarchical, composed from the unique group key
combinations

 ### disable this behavior in most cases by passing as_index=False to groupby

In [66]:
tips.groupby(['day', 'smoker'], as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Thur,Yes,19.190588,3.030000,2.352941,0.163863
1,Thur,No,17.113111,2.673778,2.488889,0.160298
2,Fri,Yes,16.813333,2.714000,2.066667,0.174783
3,Fri,No,18.420000,2.812500,2.250000,0.151650
4,Sat,Yes,21.276667,2.875476,2.476190,0.147906
5,Sat,No,19.661778,3.102889,2.555556,0.158048
6,Sun,Yes,24.120000,3.516842,2.578947,0.187250
7,Sun,No,20.506667,3.167895,2.929825,0.160113


# Apply: General split-apply-combine

### we want top 5 value of tip_pict in group

In [67]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]

In [68]:
top(tips, n=6)

,total_bill,tip,sex,smoker,day,time,size,tip_pct
109,14.31,4.00,Female,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Male,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,Male,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Female,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Female,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Male,Yes,Sun,Dinner,2,0.710345


### group by smoker, and call apply with the above function

The top function will be called on each row group from the
DataFrame, and then the results are glued together using pandas.concat,
labeling the pieces with thegroup names. T
he result therefore has a hierarchical index whose inner
level contains index values from the original DataFrame.

In [69]:
tips.groupby('smoker').apply(top)

total_bill   tip     sex smoker   day    time  size   tip_pct
smoker                                                                   
Yes    109       14.31  4.00  Female    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Male    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00  Female    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00  Female    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Male    Yes   Sun  Dinner     2  0.710345
No     88        24.71  5.85    Male     No  Thur   Lunch     2  0.236746
       185       20.69  5.00    Male     No   Sun  Dinner     5  0.241663
       51        10.29  2.60  Female     No   Sun  Dinner     2  0.252672
       149        7.51  2.00    Male     No  Thur   Lunch     2  0.266312
       232       11.61  3.39    Male     No   Sat  Dinner     2  0.291990

If you pass a function to apply that takes other arguments or keywords, you can
pass these after the
function

In [70]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

total_bill    tip     sex smoker   day    time  size  \
smoker day                                                              
Yes    Thur 197       43.11   5.00  Female    Yes  Thur   Lunch     4   
       Fri  95        40.17   4.73    Male    Yes   Fri  Dinner     4   
       Sat  170       50.81  10.00    Male    Yes   Sat  Dinner     3   
       Sun  182       45.35   3.50    Male    Yes   Sun  Dinner     3   
No     Thur 142       41.19   5.00    Male     No  Thur   Lunch     5   
       Fri  94        22.75   3.25  Female     No   Fri  Dinner     2   
       Sat  212       48.33   9.00    Male     No   Sat  Dinner     4   
       Sun  156       48.17   5.00    Male     No   Sun  Dinner     6   

                  tip_pct  
smoker day                 
Yes    Thur 197  0.115982  
       Fri  95   0.117750  
       Sat  170  0.196812  
       Sun  182  0.077178  
No     Thur 142  0.121389  
       Fri  94   0.142857  
       Sat  212  0.186220  
       Sun  156  0.103799

# Suppressing the Group Keys

resulting object has a hierarchical
index
formed from the group keys along with the indexes of each piece of the
original object. You can disable this by passing group_keys=False to groupby

In [71]:
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,sex,smoker,day,time,size,tip_pct
109,14.31,4.00,Female,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Male,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Female,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Female,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Male,Yes,Sun,Dinner,2,0.710345
88,24.71,5.85,Male,No,Thur,Lunch,2,0.236746
185,20.69,5.00,Male,No,Sun,Dinner,5,0.241663
51,10.29,2.60,Female,No,Sun,Dinner,2,0.252672
149,7.51,2.00,Male,No,Thur,Lunch,2,0.266312
232,11.61,3.39,Male,No,Sat,Dinner,2,0.291990


# Quantile and Bucket Analysis

pandas has some tools,
in particular cut and qcut, for slicing data up into buckets with bins of your
choosing or
by sample quantiles. Combining these functions with groupby makes it
convenient to perform
bucket or quantile analysis on a dataset.

In [72]:
frame = pd.DataFrame({'data1': np.random.randn(1000),
                      'data2': np.random.randn(1000)})

In [73]:
quartiles = pd.cut(frame.data1, 4)

In [74]:
quartiles[:10]

0     (-0.293, 1.393]
1    (-1.979, -0.293]
2     (-0.293, 1.393]
3     (-0.293, 1.393]
4     (-0.293, 1.393]
5      (1.393, 3.079]
6     (-0.293, 1.393]
7     (-0.293, 1.393]
8    (-3.672, -1.979]
9    (-1.979, -0.293]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-3.672, -1.979] < (-1.979, -0.293] < (-0.293, 1.393] < (1.393, 3.079]]

### The Categorical object
### returned by cut can be
### passed directly to groupby. So we
### could compute a set of statistics for the data2 column like

In [75]:
def get_stats(group):
    return {'min': group.min(), 'max': group.max(),
            'count': group.count(), 'mean': group.mean()}

In [76]:
grouped = frame.data2.groupby(quartiles)

In [77]:
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
"(-3.672, -1.979]",-2.025892,1.875787,18.0,0.164561
"(-1.979, -0.293]",-3.536014,2.786635,363.0,0.024125
"(-0.293, 1.393]",-3.034597,3.029922,548.0,0.101064
"(1.393, 3.079]",-2.013953,2.731209,71.0,-0.115592


### Return quantile numbers, pass labels=False

In [78]:
grouping = pd.qcut(frame.data1, 10, labels=False)

In [79]:
grouped = frame.data2.groupby(grouping)

In [80]:
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
0,-2.736305,1.875787,100.0,0.037809
1,-3.536014,2.754902,100.0,0.044912
2,-2.595063,2.786635,100.0,0.140218
3,-2.219753,2.527667,100.0,-0.131262
4,-2.253306,2.116839,100.0,0.008245
5,-1.780473,3.029922,100.0,0.226231
6,-2.963880,2.274063,100.0,0.117867
7,-3.034597,2.408709,100.0,0.082289
8,-2.597159,2.238293,100.0,0.113893


# Pivot table

A pivot table is a data summarization tool
frequently found in spreadsheet programs and other data
analysis software. It aggregates a table of data by one or more keys,
arranging the data in a rectangle with some of the group keys along the
rows and some along the columns

In [81]:
tips.pivot_table(index=['day', 'smoker'])  # similar to groupby

size       tip   tip_pct  total_bill
day  smoker                                          
Thur Yes     2.352941  3.030000  0.163863   19.190588
     No      2.488889  2.673778  0.160298   17.113111
Fri  Yes     2.066667  2.714000  0.174783   16.813333
     No      2.250000  2.812500  0.151650   18.420000
Sat  Yes     2.476190  2.875476  0.147906   21.276667
     No      2.555556  3.102889  0.158048   19.661778
Sun  Yes     2.578947  3.516842  0.187250   24.120000
     No      2.929825  3.167895  0.160113   20.506667

In [82]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],
                 columns='smoker')

size             tip_pct          
smoker            Yes        No       Yes        No
time   day                                         
Lunch  Thur  2.352941  2.500000  0.163863  0.160311
       Fri   1.833333  3.000000  0.188937  0.187735
Dinner Thur       NaN  2.000000       NaN  0.159744
       Fri   2.222222  2.000000  0.165347  0.139622
       Sat   2.476190  2.555556  0.147906  0.158048
       Sun   2.578947  2.929825  0.187250  0.160113

### by passing  margins=True. 
### This will add All row and column labels, 
### with corresponding values being the group statistics for all the data within a single tier
### resulting all values AS MEAN

In [83]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],
                 columns='smoker', margins=True)   

size                       tip_pct                    
smoker            Yes        No       All       Yes        No       All
time   day                                                             
Lunch  Thur  2.352941  2.500000  2.459016  0.163863  0.160311  0.161301
       Fri   1.833333  3.000000  2.000000  0.188937  0.187735  0.188765
Dinner Thur       NaN  2.000000  2.000000       NaN  0.159744  0.159744
       Fri   2.222222  2.000000  2.166667  0.165347  0.139622  0.158916
       Sat   2.476190  2.555556  2.517241  0.147906  0.158048  0.153152
       Sun   2.578947  2.929825  2.842105  0.187250  0.160113  0.166897
All          2.408602  2.668874  2.569672  0.163196  0.159328  0.160803

In [84]:
tips.pivot_table('tip_pct', index=['time', 'smoker'], columns='day',
                 aggfunc=len, margins=True)

day            Thur   Fri   Sat   Sun    All
time   smoker                               
Lunch  Yes     17.0   6.0   NaN   NaN   23.0
       No      44.0   1.0   NaN   NaN   45.0
Dinner Yes      NaN   9.0  42.0  19.0   70.0
       No       1.0   3.0  45.0  57.0  106.0
All            62.0  19.0  87.0  76.0  244.0

### Replacing Null Value

In [85]:
tips.pivot_table('tip_pct', index=['time', 'size', 'smoker'],
                 columns='day', aggfunc='mean', fill_value=0)

day                     Thur       Fri       Sat       Sun
time   size smoker                                        
Lunch  1    Yes     0.000000  0.223776  0.000000  0.000000
            No      0.181728  0.000000  0.000000  0.000000
       2    Yes     0.158843  0.181969  0.000000  0.000000
            No      0.166005  0.000000  0.000000  0.000000
       3    Yes     0.204952  0.000000  0.000000  0.000000
            No      0.084246  0.187735  0.000000  0.000000
       4    Yes     0.155410  0.000000  0.000000  0.000000
            No      0.138919  0.000000  0.000000  0.000000
       5    No      0.121389  0.000000  0.000000  0.000000
       6    No      0.173706  0.000000  0.000000  0.000000
Dinner 1    Yes     0.000000  0.000000  0.325733  0.000000
            No      0.000000  0.000000  0.137931  0.000000
       2    Yes     0.000000  0.171297  0.148668  0.207893
            No      0.159744  0.139622  0.162705  0.168859
       3    Yes     0.000000  0.000000  0.144995  0.152660
            No      0.000000  0.000000  0.154661  0.152663
       4    Yes     0.000000  0.117750  0.124515  0.193370
            No      0.000000  0.000000  0.150096  0.148143
       5    Yes     0.000000  0.000000  0.106572  0.065660
            No      0.000000  0.000000  0.000000  0.206928
       6    No      0.000000  0.000000  0.000000  0.103799

# Cross-Tabulations: Crosstab

A cross-tabulation (or crosstab for short) is a special
case of a pivot table that computes group frequencies.

In [86]:
pd.crosstab([tips.time, tips.day], tips.smoker, margins=True)

smoker       Yes   No  All
time   day                
Lunch  Thur   17   44   61
       Fri     6    1    7
Dinner Thur    0    1    1
       Fri     9    3   12
       Sat    42   45   87
       Sun    19   57   76
All           93  151  244

In [87]:
data = pd.DataFrame({'City':['Hyd','Blore','Hyd','Blore','Blore','Blore',
                                    'Hyd','Hyd','Blore','Hyd'], 
                     'Handedness' : ['Right-handed','Left-handed','Right-handed','Right-handed',
                                     'Left-handed','Right-handed','Right-handed',
                                     'Left-handed','Right-handed','Right-handed']})

In [88]:
pd.crosstab(data.City, data.Handedness, margins=True)

Handedness,Left-handed,Right-handed,All
City,,,
Blore,2,3,5
Hyd,1,4,5
All,3,7,10
